https://gpt-index.readthedocs.io/en/latest/examples/index_structs/doc_summary/DocSummary.html

In [1]:
import phoenix as px

In [2]:
session = px.launch_app()

ERROR:    [Errno 48] error while attempting to bind on address ('127.0.0.1', 6060): address already in use
Traceback (most recent call last):
  File "/Users/saazizi/miniconda3/lib/python3.10/site-packages/uvicorn/server.py", line 160, in startup
    server = await loop.create_server(
  File "uvloop/loop.pyx", line 1790, in create_server
OSError: [Errno 48] error while attempting to bind on address ('127.0.0.1', 6060): address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/saazizi/miniconda3/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/saazizi/miniconda3/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/saazizi/miniconda3/lib/python3.10/site-packages/uvicorn/server.py", line 61, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/Users/saazizi/miniconda3/lib/python3.10/asyncio/runne

In [52]:
VIDEO_ID = "transcriptions/youtube_5p248yoa3oE.json"

In [3]:
import json
import os
from getpass import getpass
from urllib.request import urlopen

import openai
import pandas as pd
import phoenix as px
from gcsfs import GCSFileSystem
from llama_index import ServiceContext, StorageContext, load_index_from_storage, set_global_handler
from llama_index.embeddings import OpenAIEmbedding
from llama_index.graph_stores.simple import SimpleGraphStore
from llama_index.llms import OpenAI
from phoenix.experimental.evals import (
    OpenAIModel,
    compute_precisions_at_k,
    run_relevance_eval,
)
from tqdm import tqdm

pd.set_option("display.max_colwidth", 1000)

In [4]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())


In [5]:
set_global_handler("arize_phoenix")

In [6]:
import os
import openai

In [7]:
import logging
import sys
import json

logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [8]:
import nest_asyncio

nest_asyncio.apply()

In [9]:
from llama_index import (
    SimpleDirectoryReader,
    ServiceContext,
    get_response_synthesizer,
)
from llama_index.indices.document_summary import DocumentSummaryIndex
from llama_index.llms import OpenAI
from llama_index.schema import Document

In [10]:
import json
from llama_index import Document

def load_json_file_and_extract_text(json_file_path):
    """Loads a JSON file and extracts the content of the "text" key.

    Args:
    json_file_path: The path to the JSON file.

    Returns:
    A list of Document objects, containing the extracted content of the "text" key.
    """

    documents = []

    with open(json_file_path, "r") as json_file:
        json_data = json.load(json_file)
        # return json_data
        documents.append(Document(doc_id=json_file_path , text=json_data["text"]))

    return documents


In [13]:
document = load_json_file_and_extract_text("transcriptions/youtube_5p248yoa3oE.json")

# Build Document Summary Index

In [14]:
# LLM (gpt-3.5-turbo)
chatgpt = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=chatgpt, chunk_size=1024)

In [15]:
# default mode of building the index
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)
doc_summary_index = DocumentSummaryIndex.from_documents(
    document,
    service_context=service_context,
    response_synthesizer=response_synthesizer,
    show_progress=True,
)
query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", use_async=True
)

Parsing documents into nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/1 [00:00<?, ?it/s]

current doc id: transcriptions/youtube_5p248yoa3oE.json


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
print(doc_summary_index.get_document_summary(doc_id='transcriptions/youtube_5p248yoa3oE.json'))

The provided text is about the topic of artificial intelligence (AI) and its current state, potential growth, and impact on various industries and society as a whole. It discusses the challenges of applying AI outside of the tech and consumer software internet sectors, as well as the trends that are making it easier for businesses in different industries to adopt AI. The text also mentions the author's venture studio, AI Fund, and their approach to building startups in the AI space. It addresses the risks and ethical considerations associated with AI, such as bias and job disruption. Additionally, the text touches on the progress made in reducing bias in AI systems, the potential disruption of jobs due to AI automation, the challenges in achieving artificial general intelligence, and the author's perspective on the risk of AI causing human extinction. Overall, the text emphasizes the opportunities and importance of building concrete use cases for AI.

Some questions that this text can 

In [17]:
# doc_summary_index.storage_context.persist("index")

In [18]:
# from llama_index.indices.loading import load_index_from_storage
# from llama_index import StorageContext

# # rebuild storage context
# storage_context = StorageContext.from_defaults(persist_dir="index")
# doc_summary_index = load_index_from_storage(storage_context)

In [20]:

def query_summary(query):
    response = query_engine.query(query)
    return response.response
r = query_summary("What are the author's thoughts on the risks and benefits of AI for humanity")
r

# Perform Retrieval from Document Summary Index

In [23]:
# query_engine = doc_summary_index.as_query_engine(
#     response_mode="tree_summarize", use_async=True
# )

In [24]:
# response = query_engine.query("bias and fairness in AI systems ")


# Index Chunks

In [25]:
from llama_index import (
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.schema import Node, Document
from llama_index.indices.document_summary import DocumentSummaryIndex


In [26]:
def json_file_document(json_file_path):
    with open(json_file_path, "r") as f:
        json_data = json.load(f)
    
    documents = []
    for segment in json_data["segments"]:
        text = segment["text"]
        start = segment["start"]
        metadata = {"start": start}
        documents.append(Document(text=text,
                                 metadata=metadata)
                        )
    return documents

In [27]:
docs = json_file_document(VIDEIO_ID)

In [28]:
from llama_index.embeddings import HuggingFaceEmbedding
import chromadb


In [29]:
# create client and a new collection
chroma_client = chromadb.EphemeralClient()
try:
    chroma_collection = chroma_client.create_collection("quickstart")
except:
    pass
# define embedding function
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [30]:
docs

[Document(id_='47549895-b45d-4b18-ac68-832ecc802c63', embedding=None, metadata={'start': 4.999999999999998}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='7b0d5cbaea75ebe4cd88b681e44129bd0181221b1f90613cb5bc7251bcd1268c', text=' It is my pleasure to welcome Dr. Andrew Wu tonight.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='0132d366-a292-4a17-bd9a-8a2fb4fb6ee2', embedding=None, metadata={'start': 11.28}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='75ff31414aef3d654c85fff4f31f4bedadc847b3d1a37445c3c71ab30457e991', text=' Andrew is the managing general partner of AI Fund, founder of Deep Learning AI, and', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='b1d11082-da71-44d0-a2

In [31]:
from llama_index.vector_stores import ChromaVectorStore


In [32]:

# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(embed_model=embed_model)
index = VectorStoreIndex.from_documents(
    docs, storage_context=storage_context, service_context=service_context,
show_progress=True,)


Parsing documents into nodes:   0%|          | 0/370 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/370 [00:00<?, ?it/s]

In [33]:
retreiver = index.as_retriever(similarity_top_k=5)

In [43]:
def search(query):
    response = retreiver.retrieve(query)
    return response

In [49]:
responses = search("What are the author's thoughts on the risks and benefits of AI for humanity")
for response in responses:
    print(response.text, response.metadata.get("start"))
    print(25*"-")

meaningful extinction risk for humanity. I think that people worry we can't control AI, 2088.7
-------------------------
real extinction risk to humanity. AI, having more intelligence, even artificial intelligence 2156.32
-------------------------
extinction risk for humanity. Candidly, I don't see it. I just don't see how AI creates any 2083.3
-------------------------
second. Just a few slides to talk about risks and social impact. So, AI is a very powerful 1881.58
-------------------------
When I think of the biggest risk of AI, I think that the biggest risk, one of the biggest 1945.46
-------------------------


In [63]:
from IPython.display import display, HTML

In [60]:
def display_youtube(video_id, start_time):
    return display(
        HTML(
            f'''<iframe 
            width="560" 
            height="315" 
            src="https://www.youtube.com/embed/{video_id}?start={start_time}" 
            frameborder="0" 
            allowfullscreen>
            </iframe>'''))


In [65]:
display_youtube(video_id, start_time=2088)

In [1]:
import torch

In [2]:
torch.cuda.is_available()

False